In [1]:
# GENERAL SCRIPT FOR ADVERSARIAL TRAINING -- COPY AND MODIFY FOR TRAINING  
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision

from torch.autograd import Variable
import numpy as np


# Universal import block 
# Block to get the relative imports working 
import os
import sys 
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import config
import matplotlib.pyplot as plt 
import prebuilt_loss_functions as plf
import loss_functions as lf 
import utils.pytorch_utils as utils
import utils.image_utils as img_utils
import cifar10.cifar_loader as cifar_loader
import cifar10.cifar_resnets as cifar_resnets
import adversarial_attacks as aa
import adversarial_training as advtrain
import adversarial_evaluation as adveval
import utils.checkpoints as checkpoints
import adversarial_perturbations as ap
import adversarial_attacks_refactor as aar 
import spatial_transformers as st
import importlib
importlib.reload(ap)

<module 'adversarial_perturbations' from '/tigress/joshuacp/projects/cifar10-human-experiments/mister_ed/adversarial_perturbations.py'>

In [2]:
# GLOBAL VARS 
DATASET = 'CIFAR10' # alternative is 'IMAGENET'
USE_GPU = False #torch.cuda.is_available()

TRAIN_BATCH_SIZE = 128 
VAL_BATCH_SIZE = 16 

CIFAR_ARCHITECTURE = 'resnet32' # If you change, change in code too
IMAGENET_ARCHITECTURE = 'nasnetalarge' 
EXPERIMENT_NAME = 'reg_adv_test_PGD_Linf8_Lr01_Iter50'

In [3]:
""" Goal here is to make sure adversarial training works under the refactored model. It _should_, but it might need 
    a few tweaks 
"""

if DATASET == 'CIFAR10':
    classifier_net, normalizer = cifar_loader.load_pretrained_cifar_resnet(flavor=32,
                                                                           use_gpu=USE_GPU,
                                                                           return_normalizer=True)
    classifier_net.eval()
    train_loader = cifar_loader.load_cifar_data('train', normalize=False, 
                                                batch_size=TRAIN_BATCH_SIZE, 
                                                use_gpu=USE_GPU)
    val_loader = cifar_loader.load_cifar_data('val', normalize=False, 
                                              batch_size=VAL_BATCH_SIZE, 
                                              use_gpu=USE_GPU)
    ARCHITECTURE = CIFAR_ARCHITECTURE
    
else:
    classifier_net, normalizer = imagenet_loader.load_pretrained_imagenet(arch=IMAGENET_ARCHITECTURE,
                                                                          use_gpu=USE_GPU, 
                                                                          return_normalizer=True)
    classifier_net.eval() 
    train_loader = imagenet_loader.load_imagenet_data('train', normalize=False, 
                                                      batch_size=TRAIN_BATCH_SIZE, 
                                                      use_gpu=USE_GPU)
    val_loader = imagenet_loader.load_imagenet_data('val', normalize=False, 
                                                    batch_size=VAL_BATCH_SIZE, 
                                                    use_gpu=USE_GPU)
    ARCHITECTURE = IMAGENET_ARCHITECTURE

examples, labels = next(iter(val_loader))
if USE_GPU:
    examples = examples.cuda()
    labels = labels.cuda() 

Files already downloaded and verified
Files already downloaded and verified


In [4]:
# BUILD ATTACK PARAMETERS HERE 
L_INF_BOUND = 8.0 / 255.0
def build_pgd_linf_attack_params():
    # Threat model first 

    delta_threat = ap.ThreatModel(ap.DeltaAddition, 
                                  ap.PerturbationParameters(lp_style='inf',
                                                            lp_bound=L_INF_BOUND, 
                                                            use_gpu=USE_GPU))
    attack_loss = plf.VanillaXentropy(classifier_net, normalizer=normalizer)
    pgd_attack = aar.PGD(classifier_net, normalizer, delta_threat, attack_loss)
    pgd_kwargs = {'num_iterations': 50,
                  'optimizer': optim.Adam, 
                  'optimizer_kwargs': {'lr': 0.01},
                  'signed': False, 
                  'verbose': False}
    params = advtrain.AdversarialAttackParameters(pgd_attack, 1.0, 
                                                  attack_specific_params={'attack_kwargs': pgd_kwargs})
    return params 


pgd_linf_attack_params = build_pgd_linf_attack_params()

In [5]:
classifier_net.train()
train_obj = advtrain.AdversarialTraining(classifier_net, normalizer, 
                                         EXPERIMENT_NAME, ARCHITECTURE)

train_loss = nn.CrossEntropyLoss() # USE A PREBUILT TRAIN LOSS FXN

In [6]:
train_obj.train(train_loader, 2, train_loss, 
                attack_parameters=pgd_linf_attack_params, 
                verbosity="snoop", 
                regularize_adv_scale=100.0,
                use_gpu=USE_GPU)

[1,     1] accuracy: (14.844, 100.000)
15.635957717895508 2.097002975642681
[1,     1] loss: 0.008866
[1,     2] accuracy: (13.281, 100.000)
15.283411979675293 2.1651610732078552
[1,     2] loss: 0.008724
[1,     3] accuracy: (10.156, 96.875)
14.808884620666504 2.251732349395752
[1,     3] loss: 0.008530
[1,     4] accuracy: (9.375, 95.312)
13.242385864257812 2.2456640377640724
[1,     4] loss: 0.007744
[1,     5] accuracy: (10.156, 95.312)
12.261381149291992 2.2727660834789276
[1,     5] loss: 0.007267
[1,     6] accuracy: (3.906, 88.281)
11.99526596069336 2.4037523195147514
[1,     6] loss: 0.007200
[1,     7] accuracy: (7.812, 88.281)
10.452958106994629 2.34291460365057
[1,     7] loss: 0.006398
[1,     8] accuracy: (4.688, 81.250)
9.397139549255371 2.4186069145798683
[1,     8] loss: 0.005908
[1,     9] accuracy: (2.344, 76.562)
9.075922966003418 2.4685703217983246
[1,     9] loss: 0.005772
[1,    10] accuracy: (3.125, 78.125)
8.186903953552246 2.4855567142367363
[1,    10] loss: 0

Process Process-5:
Process Process-7:
Process Process-6:
Process Process-8:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/joshuacp/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/joshuacp/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/joshuacp/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/joshuacp/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/joshuacp/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/joshuacp/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/joshuacp/anaconda3/lib/python3.6/site-packages

KeyboardInterrupt: 